# Battle of Neighbourhoods

## Table of Contents

1. [Introduction](#1--Introduction)
1. [Discussion and Background of the Business Problem](#2--Discussion-and-Background-of-the-Business-Problem)
1. [Data Preparation](#3--Data-Preparation)

## 1- Introduction

This notebook is prepared for the final part of the capstone project of IBM data science course. In this notebook, we will try to define a business problem, look for usable data to use Foursquare location data to compare different districts of Antalya (a city in Turkey), and figure out possible solutions to our business problem.

## 2- Discussion and Background of the Business Problem

Antalya is a touristic city located on the southwest cost of Turkey. Surrounded by Taurus Mountains on the north and Mediterranean Sea on the south, it is the fifth most populous city of Turkey with an over one million people. 
<img src="https://www.researchgate.net/publication/326379220/figure/fig1/AS:647977995624449@1531501401296/Location-of-Antalya-province-in-Turkey-red-colored-area-Map-shown-in-this-article-was.png" alt="Antalya" title="Antalya" />

Thanks to its beautiful landscape, wonderful beaches and historical background, Antalya attracts over 10 million tourist each year from all over the World. According to Turkish state statistics office, a record 12.5 million tourists passed through the city in 2014.

During touristic season, especially summer times, visitors all over the world provides a huge opportunity for souvenir shops.  Many people like to take a little something home with them as a reminder of their travels, something to show for where they have been, something they can use to look back and reminisce about the fantastic experience they had during their holiday or something that will make their loved ones happy when they return. Therefore, in this project we will try to provide best location to open a souvenir shop among Antalya’s 19 districts. We will especially concentrate on the central five districts that are closer to main province.

We will go through each step of this project and address them separately. I first outline the initial data preparation and describe future steps to start the battle of neighborhoods in Antalya.

**This analysis mainly targets the stakeholders or business owners that are planning to open a souvenir shop or start business in souvenir shop in Antalya province of Turkey.**

## 3- Data Preparation

For this project, we will get the post codes for each district in Antalya province and use Foursquare app to get nearby venues that attract more tourists. Also, we will examine nearby hotels and sight seeing places and based on this analysis we will try to make recommandations for the most suitable location for our souvenir business. Let's start by getting the location data for our target city.

### Getting District Data for Antalya

Fist of all we will try to optain district data for the Antalya Province. Lucily we have the data available on the state postal office website as a zip file on this link http://postakodu.ptt.gov.tr/. I have downloaded and arranged the data as csv format and uploaded the IBM cloud. Here, we will read this data into a pandas dataframe.

In [2]:
# The code was removed by Watson Studio for sharing.

,Province,City,District,Postcode
0,ANTALYA,AKSEKİ,AKSEKİ,7630
1,ANTALYA,AKSEKİ,AKSEKİ,7630
2,ANTALYA,AKSEKİ,AKSEKİ,7630
3,ANTALYA,AKSEKİ,AKSEKİ,7630
4,ANTALYA,AKSEKİ,AKSEKİ,7630


we need to modify Postcode column to start with 0 and convert it to type string. and also we need to remove dublicate values of Postcode since we only need the center of each district.

In [3]:
df_antalya['Postcode'] = pd.DataFrame('0'+df_antalya['Postcode'].astype(str))
df_antalya_new = df_antalya.groupby('Postcode').agg(lambda x: ','.join(set(x))).reset_index()
df_antalya_new.head()

,Postcode,Province,City,District
0,07010,ANTALYA,MURATPAŞA,ESKİSANAYİ
1,07020,ANTALYA,KEPEZ,VARSAK
2,07025,ANTALYA,KEPEZ,DOKUMA
3,07030,ANTALYA,MURATPAŞA,MELTEM
4,07040,ANTALYA,MURATPAŞA,ÇARŞI


### Get the Location Coordinates for each District
Now our Postcodes are ready. We can start getting geolocation for each district. In order to do that, we will import and install neccessary librareis first.

In [4]:
#for geolocation search
!pip install geopandas 
!pip install geopy 
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

     |████████████████████████████████| 931kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 22.9MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 50.8MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 26.8MB/s eta 0:00:01


Now we can create a new dataframe with district coordinates

In [5]:
#get the location for antalya
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Antalya, TR")

df_address = pd.DataFrame(df_antalya_new['Postcode'].astype(str) + ', Antalya, TR')

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)
# 2- - create location column
df_antalya_new['location'] = df_address['Postcode'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_antalya_new['point'] = df_antalya_new['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_antalya_new[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_antalya_new['point'].tolist(), index=df_antalya_new.index)

df_antalya_new


,Postcode,Province,City,District,location,point,latitude,longitude,altitude
0,07010,ANTALYA,MURATPAŞA,ESKİSANAYİ,"(Cumhuriyet Mahallesi, Muratpaşa, Antalya, Akd...","(36.9041492, 30.700651575000002, 0.0)",36.904149,30.700652,0.0
1,07020,ANTALYA,KEPEZ,VARSAK,"(Antalya, Muratpaşa, Antalya, Akdeniz Bölgesi,...","(36.9009641, 30.6954846, 0.0)",36.900964,30.695485,0.0
2,07025,ANTALYA,KEPEZ,DOKUMA,"(Özgürlük Mahallesi, Kepez, Antalya, Akdeniz B...","(36.91393495, 30.67963175, 0.0)",36.913935,30.679632,0.0
3,07030,ANTALYA,MURATPAŞA,MELTEM,"(Muratpaşa, Antalya, Akdeniz Bölgesi, 07030, T...","(36.8804352369126, 30.784500713480504, 0.0)",36.880435,30.784501,0.0
4,07040,ANTALYA,MURATPAŞA,ÇARŞI,"(Antalya, Muratpaşa, Antalya, Akdeniz Bölgesi,...","(36.8934522, 30.7015535, 0.0)",36.893452,30.701553,0.0
5,07050,ANTALYA,MURATPAŞA,BAHÇELİEVLER,"(Aqua, Adnan Menderes Bulvarı, Sorgun, Manavga...","(36.758141, 31.4275232, 0.0)",36.758141,31.427523,0.0
6,07060,ANTALYA,KEPEZ,YENİEMEK,"(Etiler Mahallesi, Muratpaşa, Antalya, Akdeniz...","(36.89754660654988, 30.709009476813485, 0.0)",36.897547,30.709009,0.0
7,07070,ANTALYA,KONYAALTI,ARAPSUYU,"(Konyaaltı, Antalya, Akdeniz Bölgesi, 07070, T...","(36.86811418704364, 30.619186886081952, 0.0)",36.868114,30.619187,0.0
8,07080,ANTALYA,KEPEZ,ERENKÖY,"(Kanal Mahallesi, Kepez, Antalya, Akdeniz Bölg...","(36.91818335442572, 30.67536680301581, 0.0)",36.918183,30.675367,0.0
9,07090,ANTALYA,KEPEZ,AHATLI,"(Kültür Mahallesi, Kepez, Antalya, Akdeniz Böl...","(36.9070753, 30.6622891, 0.0)",36.907075,30.662289,0.0


### Show each District on the MAP

We can pinpoint each district on the map. Let's first import the libraries.

In [6]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

Draw Districts of Antalya on the map

In [7]:
antalya_districts = folium.Map(
    location=[location.latitude,location.longitude],
    tiles='OpenStreetMap',
    zoom_start=9,
)

df_antalya_new.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(antalya_districts), axis=1)

antalya_districts